In [1]:
%load_ext autoreload
%autoreload 2
import requests
import json
import re
from pprint import pprint
import istsos4_utils as st

# base url
base_url = "http://api:5000/istsos4/v1.1/"

# Headers (optional, but specifying Content-Type ensures proper handling of JSON data)
headers = {
    'Content-Type': 'application/json'
}

meteo = st.sta(base_url)

### The Sensing Entities
The entities of the SensorThings API's Sensing component are illustrated in the following figure.

<img src="img/STA_schema.png" width="800">

To maintain proper references between elements, you must create them in a specific order:

1. Thing
2. Location
3. ObservedProperty
4. Sensor
5. Datastream
6. Observation (+FeatureOfInterest)


### Create a Thing

In [2]:
body = {
  "name" : "MAG_LOD6",
  "description" : "Maggia river in Lodrino",
  "properties" : {
    "subcatchment" : "Maggia",
    "subcatchment_area" : 4000,
    "subcatchment_number": 12345
  }
}

# POST request with the JSON body
response = requests.post(base_url + 'Things', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Thing created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Thing @iot.id using a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    thing_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Thing created successfully (http://api:5000/istsos4/v1.1/Things(1))


### Create a Location
To create a Location and link it to an existing Thing, you include the Thing's ID in the JSON payload of the Location you are creating.

In [3]:
body = {
  "name": "Location of the MAG_LOD2 gauge",
  "description": "The gauge is located close to the bridge",
  "properties": {},
  "encodingType": "application/geo+json",
  "location": {
    "type": "Point",
    "coordinates": [8.10, 50.00]
  },
  "Things": [
    { "@iot.id": thing_id}
  ]
}

# POST request with the JSON body
response = requests.post(base_url + 'Locations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Location created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Location @iot.id using a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    location_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Location created successfully (http://api:5000/istsos4/v1.1/Locations(1))


In [4]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Locations")
pprint(response.json(), indent=2)

{ '@iot.id': 1,
  '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Things(1)',
  'Datastreams@iot.navigationLink': 'http://api:5000/istsos4/v1.1/Things(1)/Datastreams',
  'HistoricalLocations@iot.navigationLink': 'http://api:5000/istsos4/v1.1/Things(1)/HistoricalLocations',
  'Locations': [ { '@iot.id': 1,
                   '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Locations(1)',
                   'description': 'The gauge is located close to the bridge',
                   'encodingType': 'application/geo+json',
                   'location': {'coordinates': [8.1, 50], 'type': 'Point'},
                   'name': 'Location of the MAG_LOD2 gauge',
                   'properties': {}}],
  'Locations@iot.navigationLink': 'http://api:5000/istsos4/v1.1/Things(1)/Locations',
  'description': 'Maggia river in Lodrino',
  'name': 'MAG_LOD6',
  'properties': { 'subcatchment': 'Maggia',
                  'subcatchment_area': 4000,
                  'subcatchment_number': 12345}}


In [5]:
things = meteo.query_api('Things','$expand=Locations')
map = meteo.map_things(things)
map

INFO:root:query_api request: http://api:5000/istsos4/v1.1/Things?$expand=Locations


### Create an Observed Property

In [6]:
body = {
  "name": "Temperature",
  "description": "Temperature",
  "properties": {},
  "definition": "http://dd.eionet.europa.eu/vocabularyconcept/aq/meteoparameter/54"
}

# POST request with the JSON body
response = requests.post(base_url + 'ObservedProperties', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"ObservedProperty created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Observed Property @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observed_property_id = int(match.group(1))
else:
    print("No number found in parentheses.")

ObservedProperty created successfully (http://api:5000/istsos4/v1.1/ObservedProperties(1))


### Create a Sensor

In [7]:
body = {
  "name": "HDT22",
  "description": "A cheap sensor that measures Temperature and Humidity",
  "properties": {},
  "encodingType": "application/pdf",
  "metadata": "https://www.sparkfun.com/datasheets/Sensors/Temperature/DHT22.pdf"
}

# POST request with the JSON body
response = requests.post(base_url + 'Sensors', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Sensor created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Sensor @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    sensor_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Sensor created successfully (http://api:5000/istsos4/v1.1/Sensors(1))


### Create a Datastream
The Datastream requires a Thing, Sensor and ObservedProperty. 

In [8]:
body = {
  "name" : "Temperature in the Kitchen",
  "description" : "The temperature in the kitchen, measured by the sensor next to the window",
  "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
  "unitOfMeasurement": {
    "name": "Degree Celsius",
    "symbol": "°C",
    "definition": "ucum:Cel"
  },
  "Thing": {"@iot.id": thing_id},
  "Sensor": {"@iot.id": sensor_id},
  "ObservedProperty": {"@iot.id": observed_property_id}
}

# POST request with the JSON body
response = requests.post(base_url + 'Datastreams', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Datastream created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    datastream_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Datastream created successfully (http://api:5000/istsos4/v1.1/Datastreams(1))


In [9]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Locations,Datastreams($expand=Sensors,ObservedProperties)")
pprint(response.json(), indent=2)

{ '@iot.id': 1,
  '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Things(1)',
  'Datastreams': [ { '@iot.id': 1,
                     '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Datastreams(1)',
                     'ObservedProperty': { '@iot.id': 1,
                                           '@iot.selfLink': 'http://api:5000/istsos4/v1.1/ObservedProperties(1)',
                                           'definition': 'http://dd.eionet.europa.eu/vocabularyconcept/aq/meteoparameter/54',
                                           'description': 'Temperature',
                                           'name': 'Temperature',
                                           'properties': {}},
                     'Sensor': { '@iot.id': 1,
                                 '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Sensors(1)',
                                 'description': 'A cheap sensor that measures '
                                                'Temperature and Humidity',
         

### Create an Observation
When creating Observations, the following additional rules apply:

1. If the phenomenonTime is not specified in the JSON payload, the server will automatically assign the current time as the phenomenonTime.
2. If the featureOfInterest is not provided, the server will generate a FeatureOfInterest based on the Location associated with the Thing from the relevant Datastream.

#### Create Observation (Datastream in the JSON and FeatureOfInterest set to the Thing's Location)

In [10]:
body = {
  "result" : 21,
  "Datastream": {"@iot.id": datastream_id}
}

# POST request with the JSON body
response = requests.post(base_url + 'Observations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Observation created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observation_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Observation created successfully (http://api:5000/istsos4/v1.1/Observations(1))


#### Create Observation (Datastream in the JSON and new FeatureOfInterest in the JSON)

In [11]:
body = {
    "result" : 22,
    "Datastream": {"@iot.id": datastream_id},
    "FeatureOfInterest": {
        "name": "A weather station.",
        "description": "A weather station.",
        "feature": {
            "type": "Point",
            "coordinates": [
                8.6,
                50.866,
            ]
        },
        "encodingType": "application/vnd.geo+json"
    },
}

# POST request with the JSON body
response = requests.post(base_url + f'Observations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Observation created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observation_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Observation created successfully (http://api:5000/istsos4/v1.1/Observations(2))


#### Create Observation (Datastream in the URL and new FeatureOfInterest in the JSON)

In [12]:
body = {
    "result" : 23,
    "FeatureOfInterest": {
        "name": "A weather station.",
        "description": "A weather station.",
        "feature": {
            "type": "Point",
            "coordinates": [
                7.6,
                50.866
            ]
        },
        "encodingType": "application/vnd.geo+json"
    },
}

# POST request with the JSON body
response = requests.post(base_url + f'Datastreams({datastream_id})/Observations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Observation created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observation_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Observation created successfully (http://api:5000/istsos4/v1.1/Observations(3))


In [13]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Locations,Datastreams($expand=Sensors,ObservedProperties,Observations($top=3;$expand=FeaturesOfInterest))")
pprint(response.json(), indent=2)

{ '@iot.id': 1,
  '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Things(1)',
  'Datastreams': [ { '@iot.id': 1,
                     '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Datastreams(1)',
                     'Observations': [ { '@iot.id': 1,
                                         '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Observations(1)',
                                         'FeatureOfInterest': { '@iot.id': 1,
                                                                '@iot.selfLink': 'http://api:5000/istsos4/v1.1/FeaturesOfInterest(1)',
                                                                'description': 'The '
                                                                               'gauge '
                                                                               'is '
                                                                               'located '
                                                                               '

In [14]:
datastreams = meteo.query_api('Datastreams')
map = meteo.map_datastreams(datastreams)
map

INFO:root:query_api request: http://api:5000/istsos4/v1.1/Datastreams


### Creating multiple related entities in one POST
It is possible to create an entities, and its relations, in one POST, by giving the full related entity in the JSON instead of only the entitiy id.

In [15]:
body = {
    "description": "thing 1",
    "name": "thing name 1",
    "properties": { "reference": "first" },
    "Locations": [
        {
            "description": "location 1",
            "name": "location name 1",
            "location": { "type": "Point", "coordinates": [7.10, 50.00] },
            "encodingType": "application/vnd.geo+json"
        }
    ],
    "Datastreams": [
        {
            "unitOfMeasurement": {
                "name": "Lumen",
                "symbol": "lm",
                "definition": "http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html/Lumen"
            },
            "description": "datastream 1",
            "name": "datastream name 1",
            "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
            "ObservedProperty": {
                "name": "Luminous Flux",
                "definition": "http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html/LuminousFlux",
                "description": "observedProperty 1"
            },
            "Sensor": {
                "description": "sensor 1",
                "name": "sensor name 1",
                "encodingType": "application/pdf",
                "metadata": "Light flux sensor"
            },
            "Observations": [
                { 
                    "result": 24,
                },
                {
                    "result": 25,
                    "FeatureOfInterest": {
                        "name": "A weather station.",
                        "description": "A weather station.",
                        "feature": {
                            "type": "Point",
                            "coordinates": [
                                6.6,
                                50.866,
                            ]
                        },
                        "encodingType": "application/vnd.geo+json"
                    },
                },
                {
                    "result": 26,
                    "FeatureOfInterest": {
                        "name": "A weather station.",
                        "description": "A weather station.",
                        "feature": {
                            "type": "Point",
                            "coordinates": [
                                7.6,
                                50.866,
                            ]
                        },
                        "encodingType": "application/vnd.geo+json"
                    },
                }
            ]
        }
    ]
}

# POST request with the JSON body
response = requests.post(base_url + 'Things', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Thing created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Thing @iot.id using a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    thing_id = int(match.group(1))
else:
    print("No number found in parentheses.")

Thing created successfully (http://api:5000/istsos4/v1.1/Things(2))


In [16]:
things = meteo.query_api('Things','$expand=Locations')
map = meteo.map_things(things)
map

INFO:root:query_api request: http://api:5000/istsos4/v1.1/Things?$expand=Locations


In [17]:
datastreams = meteo.query_api('Datastreams')
map = meteo.map_datastreams(datastreams)
map

INFO:root:query_api request: http://api:5000/istsos4/v1.1/Datastreams
